**Installing Libraries**

In [1]:
!pip install sentence-transformers beautifulsoup4
!pip install faiss-cpu
# !pip install -q -U langchain playwright sentence_transformers faiss-gpu
# !pip install -U langchain-community


In [2]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import faiss
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Load & Explore Data

In [14]:
jobs = pd.read_csv('/content/RAG System.csv', usecols=['job_title', 'description', 'requirements', 'career_level'])
jobs.head()

,job_title,description,requirements,career_level
0,Senior Developer Relations Manager,<p>Senior Developer Relations Manager page is ...,NaN,Not specified
1,Costing Manager - Cairo,"<ul>\n<li>Supervise, design and implement a co...",<ul>\n<li>Bachelor’s degree in Accounting</li>...,Manager
2,Banquet Supervisor,Mandarin Oriental Hotel GroupMandarin Oriental...,NaN,Experienced (Non-Manager)
3,Trade Finance & Credit Collection,<p><b>About Us</b></p><br><p>Alfa Laval is a l...,NaN,Not specified
4,Taste & Wellbeing Creative Marketing Associate...,Join us and celebrate the beauty of human expe...,NaN,Not specified


In [15]:
jobs.reset_index(drop=True, inplace=True)
jobs.head()

,job_title,description,requirements,career_level
0,Senior Developer Relations Manager,<p>Senior Developer Relations Manager page is ...,NaN,Not specified
1,Costing Manager - Cairo,"<ul>\n<li>Supervise, design and implement a co...",<ul>\n<li>Bachelor’s degree in Accounting</li>...,Manager
2,Banquet Supervisor,Mandarin Oriental Hotel GroupMandarin Oriental...,NaN,Experienced (Non-Manager)
3,Trade Finance & Credit Collection,<p><b>About Us</b></p><br><p>Alfa Laval is a l...,NaN,Not specified
4,Taste & Wellbeing Creative Marketing Associate...,Join us and celebrate the beauty of human expe...,NaN,Not specified


In [16]:
# Calculate the number of NaN values in each column
nan_counts = jobs.isna().sum()

# Print the results
print(nan_counts)

# Find rows with any NaN values
rows_with_nan = jobs[jobs.isna().any(axis=1)]
print("Rows with NaN values:")
print(rows_with_nan)

job_title           0
description         0
requirements    25634
career_level        0
dtype: int64
Rows with NaN values:
                                               job_title  \
0                     Senior Developer Relations Manager   
2                                     Banquet Supervisor   
3                      Trade Finance & Credit Collection   
4      Taste & Wellbeing Creative Marketing Associate...   
6                                     BMC Remedy Analyst   
...                                                  ...   
39992                                  SAP SD Specialist   
39996                               Supply Chain Manager   
39997                                Reservation Manager   
39998                            Supply Planning Officer   
39999                                    Project Planner   

                                             description requirements  \
0      <p>Senior Developer Relations Manager page is ...          NaN   
2      Man

In [17]:
# Get summary statistics
summary = jobs.describe(include='all')
print("Summary statistics:")
print(summary)

Summary statistics:
         job_title                                        description  \
count        40000                                              40000   
unique       24968                                              34135   
top     Accountant  <p> </p><ul><li>Responding to customer queries...   
freq           208                                                134   

              requirements               career_level  
count                14366                      40000  
unique               11494                          6  
top     <ul><li></li></ul>  Experienced (Non-Manager)  
freq                   278                      14471  


# Data Preparation

In [18]:
jobs = jobs.drop(columns=['requirements'])

In [19]:
# Function to clean HTML tags
def clean_html(text):
    return BeautifulSoup(text, "html.parser").get_text()

# Apply cleaning to job descriptions and requirements
jobs['description'] = jobs['description'].apply(lambda x: clean_html(str(x)))

# Display the cleaned data
jobs.head()


<ipython-input-19-1ae91600c35a>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


,job_title,description,career_level
0,Senior Developer Relations Manager,Senior Developer Relations Manager page is loa...,Not specified
1,Costing Manager - Cairo,"\nSupervise, design and implement a consistent...",Manager
2,Banquet Supervisor,Mandarin Oriental Hotel GroupMandarin Oriental...,Experienced (Non-Manager)
3,Trade Finance & Credit Collection,About UsAlfa Laval is a leading global provide...,Not specified
4,Taste & Wellbeing Creative Marketing Associate...,Join us and celebrate the beauty of human expe...,Not specified


In [21]:
# Combine job description and Job_title
jobs['combined_text'] = jobs['description'] + ' ' + jobs['job_title']

# Create embeddings for job descriptions in chunks
chunk_size = 1000  # Define chunk size
chunks = [jobs[i:i + chunk_size] for i in range(0, jobs.shape[0], chunk_size)]

# Create Embeddings from chunks

In [22]:
# Initialize the Sentence Transformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
def create_embeddings(chunk):
    chunk['embeddings'] = chunk['combined_text'].apply(lambda x: embedding_model.encode(x))
    return chunk

In [24]:
# Apply embedding creation to each chunk
chunks = [create_embeddings(chunk) for chunk in chunks]

# Concatenate chunks back into a single dataframe
data = pd.concat(chunks)
data


<ipython-input-23-c6ad91a37381>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['embeddings'] = chunk['combined_text'].apply(lambda x: embedding_model.encode(x))
<ipython-input-23-c6ad91a37381>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['embeddings'] = chunk['combined_text'].apply(lambda x: embedding_model.encode(x))
<ipython-input-23-c6ad91a37381>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,job_title,description,career_level,combined_text,embeddings
0,Senior Developer Relations Manager,Senior Developer Relations Manager page is loa...,Not specified,Senior Developer Relations Manager page is loa...,"[-0.047061484, 0.0002702253, 0.0033784653, -0...."
1,Costing Manager - Cairo,"\nSupervise, design and implement a consistent...",Manager,"\nSupervise, design and implement a consistent...","[-0.0063196504, 0.084171005, -0.07820158, 0.06..."
2,Banquet Supervisor,Mandarin Oriental Hotel GroupMandarin Oriental...,Experienced (Non-Manager),Mandarin Oriental Hotel GroupMandarin Oriental...,"[0.029390838, 0.04998568, 0.045057967, 0.06801..."
3,Trade Finance & Credit Collection,About UsAlfa Laval is a leading global provide...,Not specified,About UsAlfa Laval is a leading global provide...,"[-0.08184936, -0.048034273, -0.047277275, 0.01..."
4,Taste & Wellbeing Creative Marketing Associate...,Join us and celebrate the beauty of human expe...,Not specified,Join us and celebrate the beauty of human expe...,"[0.020883054, -0.025496112, 0.044499386, 0.028..."
...,...,...,...,...,...
39995,Credit & Banking Accountant,Opening bank facilities for contracting compan...,Experienced (Non-Manager),Opening bank facilities for contracting compan...,"[0.03521658, -0.053588018, -0.025778897, 0.017..."
39996,Supply Chain Manager,Internal Job Title: Supply Chain ManagerBusine...,Experienced (Non-Manager),Internal Job Title: Supply Chain ManagerBusine...,"[-0.050753713, -0.020308822, 0.023562765, 0.01..."
39997,Reservation Manager,"Job DescriptionCompany Description""Why work fo...",Experienced (Non-Manager),"Job DescriptionCompany Description""Why work fo...","[-0.068315364, 0.03444628, 0.024130749, 0.0296..."
39998,Supply Planning Officer,Duties & Responsibilities: Monitors status an...,Entry Level,Duties & Responsibilities: Monitors status an...,"[-0.027419504, 0.025750343, 0.014524775, 0.027..."


# Faiss Indexing

In [25]:
# Combine all embeddings into a single matrix
all_embeddings = np.vstack(data['embeddings'].values)

# Initialize FAISS index
dimension = all_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(all_embeddings)

# Save IDs for retrieval
data['id'] = range(len(data))
id_mapping = data.set_index('id').to_dict('index')

In [27]:
# Load the tokenizer and model in half-precision
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type phi3 to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct and are newly initialized: ['model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.10.mlp.gate_proj.weight', 'model.layers.10.mlp.up_proj.weight', 'model.layers.10.self_attn.k_proj.weight', 'model.layers.10.self_attn.q_proj.weight', 'model.layers.10.self_attn.v_proj.weight', 'model.layers.11.mlp.gate_proj.weight', 'model.layers.11.mlp.up_proj.weight', 'model.layers.11.self_attn.k_proj.weight', 'model.layers.11.self_attn.q_proj.weight', 'model.layers.11.self_attn.v_proj.weight', 'model.layers.12.mlp.gate_proj.weight'

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (v_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()


In [28]:
def retrieve_jobs_faiss(query, top_k=4):
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    results = [id_mapping[idx] for idx in indices[0]]
    return results

def generate_response_faiss(retrieved_jobs, query):
    job_details = [job['combined_text'] for job in retrieved_jobs]
    prompt = f"User query: {query}\nHere are some job recommendations:\n"
    for i, job in enumerate(job_details):
        prompt += f"{i+1}. {job}\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def process_query_faiss(user_query):
    retrieved_jobs = retrieve_jobs_faiss(user_query)
    response = generate_response_faiss(retrieved_jobs, user_query)
    return response

In [29]:
# Example usage
user_query = "I want to find a job in data science"
retriJobs = retrieve_jobs_faiss(user_query)
print(retriJobs)

[{'job_title': 'Remote Data Science Analyst (Python)', 'description': 'A renowned US-based client is seeking a committed Data Scientist to join their dynamic team.As part of the team, your responsibilities as a Data Scientist will include conducting detailed peer code reviews and providing valuable feedback for ongoing enhancement. You will also collaborate closely with researchers and stakeholders to ensure alignment with project specifications.Job Responsibilities:Craft Python code to explore public datasets and derive meaningful insights.Collaborate effectively with researchers to ensure projects are aligned with company goals.Produce clear and well-documented code using Jupyter notebooks for easy understanding and maintenance.Utilize public data sources like Kaggle to tackle business-related inquiries.Job Requirements:Bachelor’s/Master’s degree in Engineering, Computer Science, or equivalent practical experience.Data scientist with a minimum of 2 years of industry experience.Demons

In [37]:
top_k = 5
max_length_per_job = 200
job_details = [job['combined_text'][:max_length_per_job] for job in retriJobs[:top_k]]
job_details

['A renowned US-based client is seeking a committed Data Scientist to join their dynamic team.As part of the team, your responsibilities as a Data Scientist will include conducting detailed peer code re',
 '\xa0Job Description:\nOur client, a vanguard in the retail tech realm based in Riyadh with a global influence, is at the cusp of data-driven innovation. As they harness the power of data to drive decision',
 '\xa0Job Description:\nOur client, a vanguard in the retail tech realm based in Riyadh with a global influence, is at the cusp of data-driven innovation. As they harness the power of data to drive decision',
 'A prominent US client is seeking Data Scientists.As a Data Scientist on our team, your daily responsibilities will involve conducting thorough peer code reviews and offering insightful feedback for im']

In [38]:
# Limit the number of job details and truncate their length
prompt = f"User query: {user_query}\nHere are some job recommendations:\n"
for i, job in enumerate(job_details):
    prompt += f"{i+1}. {job}\n"

In [ ]:
# Tokenize the prompt and generate response
inputs = tokenizer(prompt, return_tensors="pt", max_length=200, truncation=True).to(device)
outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, temperature=0.7,num_beams=2, early_stopping=True)
#outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)